<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" width="140px" alt="EPFL_logo">

## Image Processing Laboratory Notebooks
---

This Jupyter Notebook is part of a series of computer laboratories that are designed
to teach image-processing programming; they are running on the EPFL's Noto server. They are the practical complement of the theoretical lectures of the EPFL's Master course 
[**MICRO-511 Image Processing I**](https://moodle.epfl.ch/course/view.php?id=522) taught by Prof. M. Unser and Prof. D. Van de Ville.

The project is funded by the Center for Digital Education and the School of Engineering. It is owned by the [Biomedical Imaging Group](http://bigwww.epfl.ch/). 
The distribution or reproduction of the notebook is strictly prohibited without the written consent of the authors.  &copy; EPFL 2025.

**Authors**: 
    [Pol del Aguila Pla](mailto:pol.delaguilapla@epfl.ch), 
    [Kay Lächler](mailto:kay.lachler@epfl.ch),
    [Alejandro Noguerón Arámburu](mailto:alejandro.nogueronaramburu@epfl.ch),
    [Yan Liu](mailto:yan.liu@epfl.ch), and
    [Daniel Sage](mailto:daniel.sage@epfl.ch).
    
---
# Lab 1.2: Fourier transform
**Released**: Thursday, September 25, 2025

**Submission deadline**: Monday, October 6, 2025, before 23:59 on [Moodle](https://moodle.epfl.ch/course/view.php?id=522)

**Grade weight**: Lab 1 (16 points), 9% of the overall grade

**Related lectures**: Chapter 1

### Student Name: 
### SCIPER: 

Double-click on this cell and fill your name and SCIPER number. Then, run the cell below to verify your identity in Noto and set the seed for random results.

In [ ]:
import getpass
# This line recovers your camipro number to mark the images with your ID
uid = int(getpass.getuser().split('-')[2]) if len(getpass.getuser().split('-')) > 2 else ord(getpass.getuser()[0])
print(f'SCIPER: {uid}')

### Imports
In the next cell we import the python libraries and load the images that we will use throughout the lab similar to the previous notebook. Run the next cell to get your notebook ready.

In [ ]:
# Configure plotting as dynamic
%matplotlib widget

# Import required packages for this lab
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
from skimage import io
from interactive_kit import imviewer as viewer

# Loading images
mandrill = io.imread("images/mandrill.tif").astype('float64')
zebra = io.imread("images/zebra.tif").astype('float64')
pens = io.imread("images/pens.tif").astype('float64')
pens_T = [io.imread(f"images/pens{i}.tif").astype('float64') for i in range(1, 6)]

# The Fourier transform (7 points)

In this second part, we will look at the 2D discrete Fourier transform (*DFT*).

You are going to:
 * understand the effects of the elements in an image on its Fourier transform (*FT*), and then
 * understand how an image is reconstructed from its FT using the inverse Fourier transform (*iFT*). 

In this part of the lab, we will only use Python. To compute the FT in Python, we will use the [`fft` module](https://numpy.org/doc/stable/reference/routines.fft.html) in NumPy, which implements the *FT* using a [fast Fourier transform (FFT)](https://en.wikipedia.org/wiki/Fast_Fourier_transform) algorithm.

Run the next cell to display the images and get familiar with them.

In [ ]:
image_list = [mandrill, pens, zebra]
imgs_viewer = viewer(image_list, widgets=True, hist=True)

# 1. Sinusoidal plane wave (2 points)

To start off this section, **for 0.5 points**, answer the MCQ below.

* Q1: What is the mathematical expression for a 2D sinusoidal plane wave, given an amplitude $A$, period $T$, phase $\phi$ and angle $\alpha$, where $\alpha$ is defined as the angle between the wave vector $(\omega_x, \omega_y)$ and the $x$-axis?

1. $s(x, y) = A\cos(2\pi T[x\sin(\alpha) + y\cos(\alpha)] + \phi)$
2. $s(x, y) = A\cos(2\pi T[x\cos(\alpha) + y\sin(\alpha)] + \phi)$
3. $s(x, y) = A\cos(\frac{2\pi}{T}[x\cos(\alpha) + y\sin(\alpha)] + \phi)$
4. $s(x, y) = A\cos(\frac{2\pi}{T}[x\sin(\alpha) + y\cos(\alpha)] + \phi)$

In the next cell, assign your choice to the variable <code>answer</code>.

In [ ]:
answer = None
# YOUR CODE HERE

In [ ]:
# Check that the answer is in the correct range
if not answer in [1, 2, 3, 4]:
    print('WARNING!!\nPossible answers are 1, 2, 3 or 4.')

In the cell below, **for 1 point**, implement the function `cos2D` that calculates $s(x,y)$ given the location (`x`, `y`) and the parameters `A`, `T` and `alpha` according to the equation you selected above.

In [ ]:
# Function that computes the 2D sinusoidal plane wave at location (x, y) given the parameters A, T and alpha
def cos2D(x, y, A, T, alpha, phi=0):
    s = 0
    
    # YOUR CODE HERE
    
    return s

Run the next cell to perform a simple sanity check.

In [ ]:
if not np.allclose(cos2D(0, 0, 0.5, 32, 0), 0.5):
    print(f'WARNING: The value at (x=0, y=0) should be 0.5 when using A=0.5, T=32 and alpha=0. Your result is {cos2D(0, 0, 0.5, 32, 0):.3f}')
if not np.allclose(cos2D(8, 16, 1, 32, 0), 0):
    print(f'WARNING: The value at (x=8, y=0) should be 0 when using A=1, T=32 and alpha=0. Your result is {cos2D(8, 0, 1, 32, 0):.3f}')
if not np.allclose(cos2D(8, 16, 0.8, 32, np.pi/2), -0.8):
    print(f'WARNING: The value at (x=8, y=16) should be -0.8 when using A=0.5, T=32 and alpha=90°. Your result is {cos2D(8, 16, 0.8, 32, np.pi/2):.3f}')
if not np.allclose(cos2D(24, 24, 0.3, 32, np.pi/2), 0):
    print(f'WARNING: The value at (x=24, y=24) should be 0 when using A=0.3, T=32 and alpha=90°. Your result is {cos2D(24, 24, 0.3, 32, np.pi/2):.3f}')
if (np.allclose(cos2D(0, 0, 0.5, 32, 0), 0.5) and np.allclose(cos2D(8, 16, 1, 32, 0), 0) 
    and np.allclose(cos2D(8, 16, 0.8, 32, np.pi/2), -0.8) and np.allclose(cos2D(24, 24, 0.3, 32, np.pi/2), 0)):
    print('Nice, your function passed the sanity check. That does not guarantee that it is correct though.')

Now, we will create an interactive viewer again, allowing you to experiment with the parameters of the 2D sinusoidal plane wave and observe the changing results. 

*Hint*: Go to `Extra Widgets` to change the parameters of the sine wave, and click on `Create 2D sin`.

In [ ]:
# Creates a 2D sinusoidal plane wave of a given size and with given parameters
def create_wave(A, T, alpha, shape=(256, 256), deg=False):
    # Convert degrees to radians
    alpha = alpha/180 * np.pi if deg else alpha
    # Apply sin2D to the whole image
    return np.fromfunction(lambda y, x: cos2D(x, y, A=A, T=T, alpha=alpha), shape=shape)

# Define sliders and button
A_slider = widgets.FloatSlider(value=1, min=0, max=2, step=0.01, description='A')
T_slider = widgets.IntSlider(value=32, min=3, max=256, step=1, description='T')
alpha_slider = widgets.IntSlider(value=0, min=-90, max=90, step=1, description=r'$\alpha [deg]$')
button = widgets.Button(description='Create 2D sin')

# Callback function that applies the vignette effect
def sin2D_callback(img):
    return create_wave(A_slider.value, T_slider.value, alpha_slider.value, deg=True)

plt.close('all')
view = viewer(create_wave(A=1, T=32, alpha=0, deg=True), title='Sinusoidal plane wave',
              new_widgets=[A_slider, T_slider, alpha_slider, button], 
              callbacks=[sin2D_callback], widgets=True, fix_range=[-1, 1])

### Multiple Choice Question

For **0.5** points, answer the following MCQ. 

* For a 2D plane wave defined in the last cell, if you double its period $T$ ($T\leftarrow 2T$) and increase $\alpha$ by an angle $-\beta$ ($\alpha\leftarrow\alpha+(-\beta$)), everything else remains the same, you will obtain roughly:

1. Half the number of the plane maxima (the white lines) in the original image, with an angular shift of $-\beta$.
2. Twice the number of the plane maxima in the original image, with an angular shift of $\beta$.
3. Half the number of the plane maxima in the original image, with an angular shift of $\beta$.
4. Twice the number of the plane maxima in the original image, with an angular shift of $-\beta$.

In the next cell, assign your choice to the variable <code>answer</code>.

In [ ]:
# Modify the variable below to reflect your choice
answer = None
# YOUR CODE HERE

In [ ]:
# Check that the answer is in the correct range
if not answer in [1, 2, 3, 4]:
    print('WARNING!!\nPossible answers are 1, 2, 3 or 4.')

# 2. Check the linearity of the Fourier transform

The Fourier transform is a linear operator. Therefore, for three images $g_1, g_2$ and $g_3$, $F\{g_1 + g_2 + g_3\} = F\{g_1\} + F\{g_2\} + F\{g_3\}$. To test this, we will use the `mandrill` image as $g_1$ and two different plane waves as $g_2$ and $g_3$. 

**Note:** To simplify the visual analysis, we will only display the *real* part of the Fourier transform, but be aware that this applies to both the real and the imaginary part.

Below is the code to define the variables and visualize them, along with their individual Fourier transforms. The `function fft_real` returns the real part of the Fourier Transform using the [`numpy.fft`](https://numpy.org/doc/stable/reference/routines.fft.html) module.
<!--
<div class="alert alert-warning">
<b>Technical note:</b> As we mentioned before, coding an FFT is out of the scope of this course. However, we will make use of the following <i>NumPy</i> functions in the wrappers <code>fft_real</code>, <code>FT</code> and <code>iFT</code> (with the last two defined in <a href='#3.-Properties-of-the-Fourier-transform-(0.5-points)'>section 3</a>):
<ul>
<li><a href='https://numpy.org/doc/stable/reference/generated/numpy.fft.fft2.html'><code>np.fft.fft2(img)</code></a> calculates the two-dimensional FT of <code>img</code>,</li>
<li><a href='https://numpy.org/doc/stable/reference/generated/numpy.fft.fftshift.html#numpy.fft.fftshift'><code>np.fft.fftshift(ft)</code></a> shifts the frequency range of the FT <code>ft</code> from $[0, \pi]$ to $[-\frac{\pi}{2}, \frac{\pi}{2}]$,</li>
<li><a href='https://numpy.org/doc/stable/reference/generated/numpy.fft.ifft2.html#numpy.fft.ifft2'><code>np.fft.ifft2(ft)</code></a> calculates the inverse FT of the two-dimensional FT <code>ft</code>.</li></ul>While learning <i>how</i> to implement a <i>Fast Fourier Transform</i> is out of the scope of this course, make sure to understand how we use the <a href='https://numpy.org/doc/stable/reference/routines.fft.html'><code>numpy.fft</code></a> module.
</div>
-->

In [ ]:
# Function that returns only the real part of the DFT of an image
def fft_real(img):
    return np.real(np.fft.fftshift(np.fft.fft2(img)))

# Create g1, which is the mandril image converted to the range [0, 256]
g1 = (mandrill - mandrill.min()) / (mandrill.max() - mandrill.min()) * 256
# Create the two plane waves
g2 = create_wave(A=256, T=4, alpha=30, deg=True)
g3 = create_wave(A=256, T=16, alpha=-30, deg=True)

plt.close('all')
view = viewer([g1, np.abs(fft_real(g1)), g2, np.abs(fft_real(g2)), g3, np.abs(fft_real(g3))], 
              title=['g1', 'real(F{g1})', 'g2', 'real(F{g2})', 'g3', 'real(F{g3})'], subplots=(3,2))

Now that we have all the necessary components, we can perform the visual test. Run the cell below to display $g_1 + g_2 + g_3$, $F\{g_1 + g_2 + g_3\}$ and $F\{g_1\} + F\{g_2\} + F\{g_3\}$. Use the `Prev` and `Next` buttons to look at the three images. Choose a suitable colormap for better visualization.

In [ ]:
added = g1 + g2 + g3
added_before_FT = fft_real(added)
added_after_FT = fft_real(g1) + fft_real(g2) + fft_real(g3)

plt.close('all')
view = viewer([added, np.abs(added_before_FT), np.abs(added_after_FT)], widgets=True, 
              title=['$g_1 + g_2 + g_3$', 
                     r'$\operatorname{real}(F\{g_1 + g_2 + g_3\})$', 
                     r'$\operatorname{real}(F\{g_1\} + F\{g_2\} + F\{g_3\})$'])

Are the two Fourier transforms identical? Let's find out by directly comparing them numerically.

In [ ]:
if np.allclose(added_before_FT, added_after_FT):
    print('Indeed, the two Fourier transforms are the same!')
else:
    print('Nope, the two Fourier transforms are different!')

# 3. Properties of the Fourier transform (0.5 points)

In the next exercise, we will explore the impact of the magnitude and phase of the Fourier transform on image reconstruction. In the next cell, we create two functions: 
  * `FT` to calculate the magnitude (in $\mathrm{dB}$) and phase of the Fourier transform
  * `iFT` to reconstruct an image using the given magnitude and phase.

Run the next cell to define these two functions and make sure that you understand every line of the code.

In [ ]:
def FT(img):
    ft = np.fft.fftshift(np.fft.fft2(img))
    return 10 * np.log10(np.abs(ft)), np.angle(ft)

def iFT(mag, ph):
    mag = np.fft.ifftshift(10 ** (mag / 10))
    img = mag * np.exp(1j * np.fft.ifftshift(ph))
    return np.fft.ifft2(img).real

To better understand what information is stored in the phase and the magnitude of the Fourier transform, we will reconstruct the image `mandrill` in three different ways using:

 * both the magnitude and phase of FT(mandrill) 
 * the magnitude of FT(mandrill) and a random phase
 * a random magnitude and the phase of FT(mandrill)

Look at the resulting reconstructed images below and answer the following MCQ.

In [ ]:
# Generate a random image
noise = np.random.rand(*mandrill.shape) * 255

# Calculate the Fourier transform of mandrill and the random image
mag, ph = FT(mandrill)
mag_n, ph_n = FT(noise)

plt.close('all')
view = viewer([mandrill, iFT(mag, ph), iFT(mag, ph_n), iFT(mag_n, ph)], subplots=(2,2),
              title=['mandrill', 'mag=mandrill, ph=mandrill', 'mag=mandrill, ph=noise', 'mag=noise, ph=mandrill'])

### Multiple choice question

What type of information of the image is stored in the **phase** of the FT that is **not stored** in the magnitude? (**0.5 points**)

1. The spacial frequencies contained in the image
2. The intensity of the image
3. The location and shape of objects in the image

Modify the variable `answer` in the next cell to reflect your choice.

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
# Sanity check
if not answer in [1, 2, 3]:
    print('WARNING!!\nPossible answers are 1, 2 or 3.')

# 4. Image transformations and the Fourier transform (2.5 points)

To illustrate some properties of the Fourier transform, we provide you with a series of images (`pens[1-5]`), which are obtained by applying a transformation to **the original image `pens`**. 

**Note**: The slider named `transformation` will allow you to choose from the $5$ transformations applied. You will see the original image `pens` with its *FT* in the first column, and the transformed image `pensn` with its *FT* in the second column. 

Use the interactive viewer below to identify the type of transformation applied to each image in the space domain and its corresponding transformation in the Fourier domain. Answer the corresponding question below the viewer.

In [ ]:
# Generate Fourier transform of all pens images
pens_T_FT = [FT(pens_T[i]) for i in range(len(pens_T))]
pens_FT = FT(pens)
# Create slider to select transformation
slider = widgets.IntSlider(min=1, max=5, value=1, description='Transformation: ', 
                           style={'description_width':'initial'}, continuous_update=False)
# Transformation change callback function
def change_func(change):
    i = change['new']
    view.original = [pens, pens_T[i-1], pens_FT[0], pens_T_FT[i-1][0], pens_FT[1], pens_T_FT[i-1][1]]
    view.reset({'new':True})
    titles = ['pens', f'pens{i}', 
              r'$\|F\{pens\}\|$', r'$\|F\{pens' + str(i) + r'\}\|$', 
              r'$\phi(F\{pens\})$', r'$\phi(F\{pens' + str(i) + r'\})$']
    for i, ax in enumerate(view.axs):
        ax.set_title(titles[i])

# Link slider and callback function and display slider
slider.observe(change_func, names='value')
display(slider)
# Initialize viewer
plt.close('all')
view = viewer([pens, pens_T[0], pens_FT[0], pens_T_FT[0][0], pens_FT[1], pens_T_FT[0][1]], subplots=(3,2), 
              title=['pens', 'pens1', r'$\|F\{pens\}\|$', r'$\|F\{pens1\}\|$', r'$\phi(F\{pens\})$', r'$\phi(F\{pens1\})$'])

### Multiple choice questions
For a **total of 2.5 points**, answer the following two questions. Each question consists of 5 subquestions, each worth **0.25**. 

* Q1: Assign for each image (`pens[1-5]`) one of the following transformations **in the space domain**.

1. Identity
2. Scaling
3. Vertical mirror
4. Horizontal mirror
5. Rotation
6. Shift
7. Shear
8. Blur
9. Multiplication by a sinusoid


Assign each of the answer variables `answer_pens[1-5]` to one of the given transforms.



In [ ]:
# Assign the transform in the space domain to the corresponding image
answer_pens1 = None
answer_pens2 = None
answer_pens3 = None
answer_pens4 = None
answer_pens5 = None

# YOUR CODE HERE

In [ ]:
# Check that the answer is in the correct range
if not answer_pens1 in range(1, 10): 
    print('WARNING!!\nPossible values are 1, 2, 3, 4, 5, 6, 7, 8 or 9.')

In [ ]:
# Check that the answer is in the correct range
if not answer_pens2 in range(1, 10): 
    print('WARNING!!\nPossible values are 1, 2, 3, 4, 5, 6, 7, 8 or 9.')

In [ ]:
# Check that the answer is in the correct range
if not answer_pens3 in range(1, 10): 
    print('WARNING!!\nPossible values are 1, 2, 3, 4, 5, 6, 7, 8 or 9.')

In [ ]:
# Check that the answer is in the correct range
if not answer_pens4 in range(1, 10): 
    print('WARNING!!\nPossible values are 1, 2, 3, 4, 5, 6, 7, 8 or 9.')

In [ ]:
# Check that the answer is in the correct range
if not answer_pens5 in range(1, 10): 
    print('WARNING!!\nPossible values are 1, 2, 3, 4, 5, 6, 7, 8 or 9.')

* Q2: Assign for each image (`pens[1-5]`) one of the following image transformations **in the Fourier domain**.

1. Identity
2. Modulation
3. Shift
4. Scaling
5. Rotation
6. Low-pass filtering (low frequencies are preserved and high frequencies are discarded)
7. High-pass filtering (low frequencies are preserved and high frequencies are discarded)


Assign each of the answer variables <code>answer_pens_Fourier[1-5]</code> to one of the given trasforms.


In [ ]:
# Assign the transform in the Fourier domain to the corresponding image
answer_pens1_Fourier = None
answer_pens2_Fourier = None
answer_pens3_Fourier = None
answer_pens4_Fourier = None
answer_pens5_Fourier = None

# YOUR CODE HERE

In [ ]:
# Check that the answer is in the correct range
if not answer_pens1_Fourier in range(1, 7): 
    print('WARNING!!\nPossible values are 1, 2, 3, 4, 5, 6, 7.')

In [ ]:
# Check that the answer is in the correct range
if not answer_pens2_Fourier in range(1, 7): 
    print('WARNING!!\nPossible values are 1, 2, 3, 4, 5, 6, 7.')

In [ ]:
# Check that the answer is in the correct range
if not answer_pens3_Fourier in range(1, 7): 
    print('WARNING!!\nPossible values are 1, 2, 3, 4, 5, 6, 7.')

In [ ]:
# Check that the answer is in the correct range
if not answer_pens4_Fourier in range(1, 7): 
    print('WARNING!!\nPossible values are 1, 2, 3, 4, 5, 6, 7.')

In [ ]:
# Check that the answer is in the correct range
if not answer_pens5_Fourier in range(1, 7): 
    print('WARNING!!\nPossible values are 1, 2, 3, 4, 5, 6, 7.')

# 5. Fourier reconstruction
## 5.A. Reconstruction error (1 point)

How many Fourier coefficients do we really need to keep to have the basic information present in an image? Do all coefficients contribute the same? To quantify the quality of a certain reconstruction, we will use a metric to assess its goodness.

The metric we use is the normalized mean square error (NMSE) in dB. It calculates the difference between an image $f$ of size $K\times L$ and its reconstruction $g$, and normalizes by the total power of $f$.

$$\operatorname{NMSE}_f(g) = \frac{\sum_{k=1}^{K} \sum_{l=1}^L (g[k,l] - f[k,l])^2}{\sum_{m=1}^{K} \sum_{n=1}^L f[m,n]^2}, \qquad \operatorname{NMSE}_f(g)~[\mathrm{dB}] = 10 \log_{10}\left(\operatorname{NMSE}_f(g)\right).$$

This makes it easier for one to observe, for example, when the error has doubled ($+3~\mathrm{dB}$) or halved ($-3~\mathrm{dB}$) in plots.

For **1 point**, complete the function `nmse(f, g)` in the cell below according to the equation given above, where `f` and `g` are two NumPy arrays of the same shape.

**Warning**: Remember that your function should **not use `for` loops** to iterate through images (this will give you $0$ points), and should work for NumPy arrays of any shape.

In [ ]:
# Function that calculates the Normalized Mean Square Error in dB
def nmse(f, g):
    output = None
    
    # YOUR CODE HERE
    
    return output

In [ ]:
# Sanity check (do not worry about the divide by zero note)
if  nmse(pens, pens) != -np.inf: 
    print('The error between two equal images should be zero. In dB -infinity.')
# Check your function on the hrct image
elif nmse(pens, 0) != 0:
    print('The error of any image and a zero-image should be 1. In dB, 0.')
else:
    print('Nice, your function seems to work! Do not worry about the divide by zero warning!')

## 5.B. Fourier components (1 point)

In this section, we examine the reconstruction process of an image from a subset of its Fourier components. This touches a topic that will continue to appear in IP1 and IP2: how much does a given transform compress an image? 

We define a function `clip_fourier(img, percent)` that reconstructs an image for only `percent`$\%$ of its Fourier coefficients. If `largest=True`, only the largest are kept, while 
if `largest=False`, they are excluded, and only all the rest are kept. This will illustrate the uneven distribution of information contained in the Fourier components.

Run the next cell to define the function `clip_fourier`.

In [ ]:
 def clip_fourier(img, percent, largest=True, perc=True):
    # Get number of coefficients to keep
    if perc:
        n = np.round(img.size * percent / 100 ).astype(int)
    else: 
        n = percent
    img_ft = np.fft.fft2(img)
    # Get the threshold value. To do this, we order the Fourier coefficients 
    # from low to high and select the n-to-last ([-n]) coefficients
    threshold = np.sort(np.abs(img_ft.flatten()))[-n]
    # Get the inverse Fourier transform of the thresholded Fourier transform
    if largest == True:
        clipped_ift = np.real(np.fft.ifft2((np.abs(img_ft) >= threshold) * img_ft))
    else:
        clipped_ift = np.real(np.fft.ifft2((np.abs(img_ft) < threshold) * img_ft))
    return clipped_ift

Let's use the error metric `nmse` defined before to illustrate the difference in information contained in the few largest Fourier components compared to the information contained in the rest. In the cell below, we will reconstruct the image `zebra` using: 
 * only the `percent` largest Fourier components, and 
 * using the `100-percent` smallest components. 

Then we will compare the reconstruction error by applying the `nmse` function defined above with both reconstructions. Run the cell below to see the different reconstruction errors. Play with the variable `percent` and see what happens.

In [ ]:
import warnings
# Suppress traitlets deprecation warning - do not modify
warnings.simplefilter("ignore")

percent = 20
# First, reconstruct zebra using the largest components
zebra_largest = clip_fourier(zebra, percent)
# Reconstruct zebra using the smallest components
zebra_smallest = clip_fourier(zebra, percent, largest=False)
# Calculate the errors
error_l = nmse(zebra,zebra_largest)
error_s = nmse(zebra,zebra_smallest)
# Compare the error
print(f'The reconstruction error when using the {percent}% largest  components: NMSE = {error_l:.4f}')
print(f"The reconstruction error when using the {100 - percent}% smallest components: NMSE = {error_s:6.4f}")
# Visualize images
plt.close('all')
view = viewer([zebra, zebra_largest, zebra_smallest], 
              title=['Original', f'{percent}% Largest Components', f'{100-percent}% Smallest Components'], 
              widgets=True)

In the long run, this type of characteristics of transforms are explored using graphs like the one below, where the NMSE can be seen as a function of the percentage of the largest coefficients kept.

In [ ]:
plt.close("all")
# Create figure
fig = plt.figure(num=f"SCIPER: {uid}",figsize=(8,5));
# Plot the NMSE vs kept coefficients curve
plt.plot(np.arange(.5, 100, 5), [nmse(zebra, clip_fourier(zebra, perc)) for perc in np.arange(.5, 100, 5)], 'bo-');
# Labels and titles for clear plotting
plt.xlabel("Percentage of coefficients kept"); plt.ylabel("NMSE [dB]"); 
plt.xticks([0, 20, 40, 60, 80, 100], [f"{perc}%" for perc in [0, 20, 40, 60, 80, 100]]);
plt.title("Reconstructing zebra with the largest Fourier coefs.")
plt.show()

Now, we will create a widget to apply the function to an image and dynamically visualize its effect. 

We will define a slider to choose an integer `n` such that **the number of the largest coefficients kept is $2^n$**. 

**Note**: This is because the visual difference between the reconstructions is only apparent for percentages between $0\%$ and $2\%$, and very small steps would be needed. Keep in mind that you are no longer working with percentages.

We will also provide a checkbox to switch between the two modes of operation (keeping the largest or keeping all the rest). Click the button `Apply` to apply `clip_fourier()` with the chosen parameter to the image. Run the next cell and click on `Extra Widgets` to use the widget. Explore the results carefully.

In [ ]:
# Declare slider and checkbox
n_slider = widgets.IntSlider(value=16, min=0, max=16, step=1, description='n')
checkbox = widgets.Checkbox(value=True, description='Use largest components')

# Declare button
button = widgets.Button(description='Apply')

# Declare the button callback
def button_callback(image):
    n      = n_slider.value
    check  = checkbox.value
    output = clip_fourier(image, 2**n, largest=check, perc=False)
    return output

# Visualize
plt.close('all')
cfourier_viewer = viewer(zebra, title="Clipping the FT", new_widgets=[n_slider, checkbox, button], callbacks=[button_callback], widgets=True, normalize=True)

### Multiple Choice Question
Congratulations! You've reached the end of the notebook. Now you just need to answer these last two MCQ questions (**0.5 points** each).

* Q1: How many of the largest Fourier coefficients are required to start clearly seeing a zebra shape in the image?
    1. $2^{1}$
    2. $2^{4}$
    3. $2^{11}$
    4. $2^{16}$

Modify the variable `answer` to reflect your choice. 

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
if not answer in [1, 2, 3, 4]:
    print('WARNING!!\nPossible answers are 1, 2, 3 or 4.')

* Q2: How is it possible to reconstruct a non-periodic object such as the zebra (there is only one zebra in the image) in an image from only periodic components?
    1. The black and white stripes in the zebra make it possible.
    2. The FT assumes that the image is periodic in space.
    3. The biggest components of the FT are non-periodic, to account for such features in an image.

Modify the variable `answer` to reflect your choice. 

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
if not answer in [1, 2, 3]:
    print('WARNING!!\nPossible answers are 1, 2 or 3.')

🎉 Congratulations on finishing the second part of the Pixel-Fourier lab! 🎉

Make sure to save your notebook (you might want to keep a copy on your personal computer) and upload it to [Moodle](https://moodle.epfl.ch/course/view.php?id=522), in a zip file with the other notebook of this lab.

* Keep the name of the notebook as: *2_fourier_transform.ipynb*,
* Name the zip file: *pixel_fourier_lab.zip*.